<a href="https://colab.research.google.com/github/fernandosagastume/SPLABS/blob/master/Laboratorio7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals
# Install TensorFlow
import tensorflow as tf
#import sys
#sys.path.append('/content/drive/My Drive/SP1/')

#import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
from google.colab.patches import cv2_imshow

TensorFlow 2.x selected.


In [0]:
class ConvAutoencoder:
  @staticmethod
  def build(width, height, depth, filters=(64,128,256), latentDim=32):
    # initialize the input shape to be "channels last" along with
    # the channels dimension itself
    # channels dimension itself
    inputShape = (height, width, depth)
    chanDim = -1
    # define the input to the encoder
    inputs = Input(shape=inputShape)
    x = inputs
    # loop over the number of filters
    for f in filters:
      # apply a CONV => RELU => BN operation
      x = Conv2D(f, (3, 3), strides=1, padding="same")(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chanDim)(x)
    # flatten the network and then construct our latent vector
    volumeSize = K.int_shape(x)
    print("volumeSize:",volumeSize)
    x = Flatten()(x)
    print("x shape", K.int_shape(x))
    latent = Dense(latentDim)(x)
    # build the encoder model
    encoder = Model(inputs, latent, name="encoder")
    # start building the decoder model which will accept the
    # output of the encoder as its inputs
    latentInputs = Input(shape=(latentDim,))
    x = Dense(np.prod(volumeSize[1:]))(latentInputs)
    print("prod shape:",np.prod(volumeSize[1:]))
    print("x shape",K.int_shape(x))
    x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
    print("x shape",K.int_shape(x))
    # loop over our number of filters again, but this time in
    # reverse order
    for f in filters[::-1]:
      # apply a CONV_TRANSPOSE => RELU => BN operation
      x = Conv2DTranspose(f, (3, 3), strides=1, padding="same")(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chanDim)(x)
    # apply a single CONV_TRANSPOSE layer used to recover the
    # original depth of the image
    x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
    outputs = Activation("sigmoid")(x)
    # build the decoder model
    decoder = Model(latentInputs, outputs, name="decoder")
    # our autoencoder is the encoder + decoder
    autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
    # return a 3-tuple of the encoder, decoder, and autoencoder
    return (encoder, decoder, autoencoder)

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2

In [0]:
def build_unsupervised_dataset(data, labels, validLabel=1,
  anomalyLabel=3, contam=0.01, seed=42):
    # Shapes of training set
    print("Training set (images) shape: {shape}".format(shape=data.shape))
    # Shapes of test set
    print("Training labels (images) shape: {shape}".format(shape=labels.shape))
    # grab all indexes of the supplied class label that are *truly*
    # that particular label, then grab the indexes of the image
    # labels that will serve as our "anomalies"
    validIdxs = np.where(labels == validLabel)[0]
    anomalyIdxs = np.where(labels == anomalyLabel)[0]
    # randomly shuffle both sets of indexes
    random.shuffle(validIdxs)
    random.shuffle(anomalyIdxs)
    # compute the total number of anomaly data points to select
    i = int(len(validIdxs) * contam)
    anomalyIdxs = anomalyIdxs[:i]
    # use NumPy array indexing to extract both the valid images and
    # "anomlay" images
    validImages = data[validIdxs]
    anomalyImages = data[anomalyIdxs]
    # stack the valid images and anomaly images together to form a
    # single data matrix and then shuffle the rows
    images = np.vstack([validImages, anomalyImages])
    np.random.seed(seed)
    np.random.shuffle(images)
    # return the set of images
    return images

In [0]:
def visualize_predictions(decoded, gt, samples=10):
  # initialize our list of output images
  outputs = None
  # loop over our number of output samples
  for i in range(0, samples):
    # grab the original image and reconstructed image
    original = (gt[i] * 255).astype("uint8")
    recon = (decoded[i] * 255).astype("uint8")
  # stack the original and reconstructed image side-by-side
  output = np.hstack([original, recon])
  # if the outputs array is empty, initialize it as the current
  # side-by-side image display
  if outputs is None:
    outputs = output
  # otherwise, vertically stack the outputs
  else:
    outputs = np.vstack([outputs, output])
  # return the output images
  return outputs

In [0]:
from tensorflow import keras
# load the MNIST dataset
print("loading Fashion MNIST dataset...")
fashion_mnist = keras.datasets.fashion_mnist
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
#trainX = trainX.reshape(-1, 28, 28, 1)
#testX = testX.reshape(-1, 28, 28, 1)

loading Fashion MNIST dataset...


In [0]:
# Shapes of training set
print("Training set (images) shape: {shape}".format(shape=trainX.shape))
# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=testX.shape))

Training set (images) shape: (60000, 28, 28)
Test set (images) shape: (10000, 28, 28)


In [0]:
# build our unsupervised dataset of images with a small amount of
# contamination (i.e., anomalies) added into it
print("creating unsupervised dataset...")
images = build_unsupervised_dataset(trainX, trainY, validLabel=1,
    anomalyLabel=3, contam=0.01)

# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
images = np.expand_dims(images, axis=-1)
images = images.astype("float32") / 255.0
images.shape

creating unsupervised dataset...
Training set (images) shape: (60000, 28, 28)
Training labels (images) shape: (60000,)


(6060, 28, 28, 1)

In [0]:
# construct the training and testing split
(trainX, testX) = train_test_split(images, test_size=0.2,
    random_state=42)
trainX.shape, testX.shape

((4848, 28, 28, 1), (1212, 28, 28, 1))

In [0]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 64
# construct our convolutional autoencoder
print("building autoencoder...")
(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mean_squared_error", optimizer=opt)

building autoencoder...
volumeSize: (None, 28, 28, 256)
x shape (None, 200704)
prod shape: 200704
x shape (None, 200704)
x shape (None, 28, 28, 256)


In [0]:
np.max(trainX), np.max(testX)

(1.0, 1.0)

In [0]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 32)                6794016   
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         7584513   
Total params: 14,378,529
Trainable params: 14,376,737
Non-trainable params: 1,792
_________________________________________________________________


In [0]:
# train the convolutional autoencoder
H = autoencoder.fit(
trainX, trainX, 
validation_data=(testX, testX),
epochs=5,
batch_size=256)

Train on 4848 samples, validate on 1212 samples
Epoch 1/5
4848/4848 [==============================] - 661s 136ms/sample - loss: 0.0477 - val_loss: 0.1000
Epoch 2/5
4848/4848 [==============================] - 654s 135ms/sample - loss: 0.0361 - val_loss: 0.0571
Epoch 3/5
4848/4848 [==============================] - 654s 135ms/sample - loss: 0.0280 - val_loss: 0.0919
Epoch 4/5
4848/4848 [==============================] - 650s 134ms/sample - loss: 0.0228 - val_loss: 0.0993
Epoch 5/5
4848/4848 [==============================] - 657s 135ms/sample - loss: 0.0182 - val_loss: 0.1136


In [0]:
import sys
sys.path.append('/content/drive/My Drive/SP1/Lab7/')
#use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
print("making predictions...")
decoded = autoencoder.predict(testX)
vis = visualize_predictions(decoded, testX)
cv2.imwrite("/content/drive/My Drive/SP1/Lab7/autoencoder/encoder_vis.png", vis)

making predictions...


False

In [29]:
# construct a plot that plots and saves the training history
loss = H.history['loss']
val_loss = H.history['val_loss']
epochs = range(200)
plt.figure()

<Figure size 432x288 with 0 Axes>